In [60]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd


In [61]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A.csv")
pi = pd.read_csv("pi_2.csv")

A = np.array(A)
pi = np.array(pi)
pi = pi.T
b = np.array([1000 for _ in range(m)])  

In [62]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)
prob.solve()
opv1 = prob.value

In [63]:
def u(s,w):
    value = 0
    for i in range(m):
        value += w/m * cp.log(s[i])
    return value

In [64]:
def dynamicLearning(start, k, xp, xk, w):
	A1 = A[:m, :k]
	pi1 = pi[:,:k]
	x = cp.Variable(k)
	s = cp.Variable(m)
	# print(x.shape)
	obj = cp.Maximize(pi1@x + u(s,w))
	cons = [A1@x + s <= k/n * b, x>=0, x<=1]
	prob = cp.Problem(obj, cons)
	prob.solve(solver="SCS")
	P = prob.constraints[0].dual_value
	# print(P)
	AT = A.T

	xpNow = xp
	xkNow = xk
	for i in range(start, 2*k):
		if i >= n:
			# print(xkNow)
			global xTrue
			xTrue = xkNow
			return xkNow
		
		cri = (AT[i].dot(P))
		if(pi[0][i] > cri):
			xpNow.append(1)
		else:
			xpNow.append(0)

		test = 1
		for j in range(m):
			temp1 = A[j,i]*xpNow[i]
			temp2 = b[j] - (A[j:j+1, :i]).dot(np.array(xkNow))
			# print(i)
			# print(len(xkNow))
			if temp1 > temp2:
				test = 0
				break
		if test == 1:
			xkNow.append(xpNow[i])
		else:
			xkNow.append(0)

	dynamicLearning(2*k, 2*k, xpNow, xkNow, w)
	return xTrue
		

In [65]:
for i in range(1,21):
    w = i
    xk = [0]*50
    xp = [0]*50
    l_decision = np.array(dynamicLearning(50, 50, xp, xk, w))
    opv2 = (np.dot(pi,l_decision)[0])
    print("when w =", w)
    print("the competetive ratio is", opv2/opv1)

when w = 1
the competetive ratio is 0.978601456837532
when w = 2
the competetive ratio is 0.9782351605652236
when w = 3
the competetive ratio is 0.9783264725544019
when w = 4
the competetive ratio is 0.9777789615658169
when w = 5
the competetive ratio is 0.9782048763772149
when w = 6
the competetive ratio is 0.9778269529973068
when w = 7
the competetive ratio is 0.9779359523875596
when w = 8
the competetive ratio is 0.9780411875586937
when w = 9
the competetive ratio is 0.9780147928364383
when w = 10
the competetive ratio is 0.978833635415891
when w = 11
the competetive ratio is 0.9788856729038952
when w = 12
the competetive ratio is 0.9796775096986613
when w = 13
the competetive ratio is 0.9796020061908668
when w = 14
the competetive ratio is 0.9805342841342056
when w = 15
the competetive ratio is 0.9805787033152006
when w = 16
the competetive ratio is 0.981413721956705
when w = 17
the competetive ratio is 0.9816809764912773
when w = 18
the competetive ratio is 0.9813585028959266
when